In [1]:
import cv2
import numpy as np
import random as rng


w = 800
h = 600


cap = cv2.VideoCapture(0)

cap.set(3, w)
cap.set(4, h)

if cap.isOpened():
    ret, frame = cap.read()
else:
    ret = False

ret, frame1 = cap.read()
ret, frame2 = cap.read()


while ret:

    d = cv2.absdiff(frame1, frame2)

    grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(grey, (5, 5), 0)

    ret, th = cv2.threshold( blur, 20, 255, cv2.THRESH_BINARY)

    dilated = cv2.dilate(th, np.ones((3, 3), np.uint8), iterations=1 )

    eroded = cv2.erode(dilated, np.ones((3, 3), np.uint8), iterations=1 )

    contours, h = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cv2.drawContours(frame1, contours, -1, (0, 0, 255), 2)

    
    # Find the convex hull object for each contour
    hull_list = []
    for i in range(len(contours)):
        hull = cv2.convexHull(contours[i])
        hull_list.append(hull)
    # Draw contours + hull results
    drawing = np.zeros((eroded.shape[0], eroded.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv2.drawContours(drawing, contours, i, color)
        cv2.drawContours(drawing, hull_list, i, color)
#     cnt = contours   
#     hull = cv2.convexHull(cnt,returnPoints = False)
#     defects = cv2.convexityDefects(cnt,hull)
    
#     for i in range(defects.shape[0]):
#         s,e,f,d = defects[i,0]
#         start = tuple(cnt[s][0])
#         end = tuple(cnt[e][0])
#         far = tuple(cnt[f][0])
#         cv2.line(drawing,start,end,[0,255,0],2)
#         cv2.circle(drawing,far,5,[0,0,255],-1)

    cv2.imshow("X", drawing)
    cv2.imshow("Original", frame2)
    cv2.imshow("Output", frame1)
    if cv2.waitKey(1) == 27: # exit on ESC
        break

    frame1 = frame2
    ret, frame2 = cap.read()

cv2.destroyAllWindows()
cap.release()

In [14]:
import cv2
import numpy as np

hand_hist = None
traverse_point = []
total_rectangle = 9
hand_rect_one_x = None
hand_rect_one_y = None

hand_rect_two_x = None
hand_rect_two_y = None


def rescale_frame(frame, wpercent=130, hpercent=130):
    width = int(frame.shape[1] * wpercent / 100)
    height = int(frame.shape[0] * hpercent / 100)
    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)


def contours(hist_mask_image):
    gray_hist_mask_image = cv2.cvtColor(hist_mask_image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_hist_mask_image, 0, 255, 0)
    cont, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return cont

def draw_rect(frame):
    rows, cols, _ = frame.shape
    global total_rectangle, hand_rect_one_x, hand_rect_one_y, hand_rect_two_x, hand_rect_two_y

    hand_rect_one_x = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    hand_rect_one_y = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    hand_rect_two_x = hand_rect_one_x + 10
    hand_rect_two_y = hand_rect_one_y + 10

    for i in range(total_rectangle):
        cv2.rectangle(frame, (hand_rect_one_y[i], hand_rect_one_x[i]),
                      (hand_rect_two_y[i], hand_rect_two_x[i]),
                      (0, 255, 0), 1)

    return frame


def hand_histogram(frame):
    global hand_rect_one_x, hand_rect_one_y

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    roi = np.zeros([90, 10, 3], dtype=hsv_frame.dtype)

    for i in range(total_rectangle):
        roi[i * 10: i * 10 + 10, 0: 10] = hsv_frame[hand_rect_one_x[i]:hand_rect_one_x[i] + 10,
                                          hand_rect_one_y[i]:hand_rect_one_y[i] + 10]

    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)


def hist_masking(frame, hist):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)

    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
    cv2.filter2D(dst, -1, disc, dst)

    ret, thresh = cv2.threshold(dst, 150, 255, cv2.THRESH_BINARY)

    # thresh = cv2.dilate(thresh, None, iterations=5)

    thresh = cv2.merge((thresh, thresh, thresh))

    return cv2.bitwise_and(frame, thresh)


def centroid(max_contour):
    moment = cv2.moments(max_contour)
    if moment['m00'] != 0:
        cx = int(moment['m10'] / moment['m00'])
        cy = int(moment['m01'] / moment['m00'])
        return cx, cy
    else:
        return None


def farthest_point(defects, contour, centroid):
    if defects is not None and centroid is not None:
        s = defects[:, 0][:, 0]
        cx, cy = centroid

        x = np.array(contour[s][:, 0][:, 0], dtype=np.float)
        y = np.array(contour[s][:, 0][:, 1], dtype=np.float)

        xp = cv2.pow(cv2.subtract(x, cx), 2)
        yp = cv2.pow(cv2.subtract(y, cy), 2)
        dist = cv2.sqrt(cv2.add(xp, yp))

        dist_max_i = np.argmax(dist)

        if dist_max_i < len(s):
            farthest_defect = s[dist_max_i]
            farthest_point = tuple(contour[farthest_defect][0])
            return farthest_point
        else:
            return None


def draw_circles(frame, traverse_point):
    if traverse_point is not None:
        for i in range(len(traverse_point)):
            cv2.circle(frame, traverse_point[i], int(5 - (5 * i * 3) / 100), [0, 255, 255], -1)


def manage_image_opr(frame, hand_hist):
    hist_mask_image = hist_masking(frame, hand_hist)

    hist_mask_image = cv2.erode(hist_mask_image, None, iterations=2)
    hist_mask_image = cv2.dilate(hist_mask_image, None, iterations=2)

    contour_list = contours(hist_mask_image)
    max_cont = max(contour_list, key=cv2.contourArea)

    cnt_centroid = centroid(max_cont)
    cv2.circle(frame, cnt_centroid, 5, [255, 0, 255], -1)

    if max_cont is not None:
        hull = cv2.convexHull(max_cont, returnPoints=False)
        defects = cv2.convexityDefects(max_cont, hull)
        far_point = farthest_point(defects, max_cont, cnt_centroid)
        print("Centroid : " + str(cnt_centroid) + ", farthest Point : " + str(far_point))
        cv2.circle(frame, far_point, 5, [0, 0, 255], -1)
        if len(traverse_point) < 20:
            traverse_point.append(far_point)
        else:
            traverse_point.pop(0)
            traverse_point.append(far_point)

        draw_circles(frame, traverse_point)


def main():
    global hand_hist
    is_hand_hist_created = False
    capture = cv2.VideoCapture(0)

    while capture.isOpened():
        pressed_key = cv2.waitKey(1)
        _, frame = capture.read()
        frame = cv2.flip(frame, 1)

        if pressed_key & 0xFF == ord('z'):
            is_hand_hist_created = True
            hand_hist = hand_histogram(frame)

        if is_hand_hist_created:
            manage_image_opr(frame, hand_hist)

        else:
            frame = draw_rect(frame)

        cv2.imshow("Live Feed", rescale_frame(frame))

        if pressed_key == 27:
            break

    cv2.destroyAllWindows()
    capture.release()


if __name__ == '__main__':
    main()

Centroid : (255, 231), farthest Point : (639, 419)
Centroid : (264, 231), farthest Point : (639, 419)
Centroid : (258, 231), farthest Point : (639, 419)
Centroid : (262, 233), farthest Point : (639, 419)
Centroid : (263, 226), farthest Point : (639, 419)
Centroid : (283, 230), farthest Point : (639, 419)
Centroid : (263, 235), farthest Point : (639, 419)
Centroid : (265, 231), farthest Point : (639, 419)
Centroid : (264, 232), farthest Point : (639, 419)
Centroid : (261, 229), farthest Point : (639, 419)
Centroid : (255, 229), farthest Point : (639, 419)
Centroid : (266, 232), farthest Point : (639, 419)
Centroid : (254, 235), farthest Point : (639, 419)
Centroid : (254, 230), farthest Point : (639, 419)
Centroid : (257, 237), farthest Point : (639, 419)
Centroid : (269, 237), farthest Point : (639, 419)
Centroid : (267, 230), farthest Point : (639, 419)
Centroid : (257, 234), farthest Point : (639, 419)
Centroid : (272, 229), farthest Point : (639, 419)
Centroid : (262, 231), farthest

Centroid : (282, 231), farthest Point : (639, 419)
Centroid : (282, 231), farthest Point : (639, 419)
Centroid : (297, 235), farthest Point : (639, 419)
Centroid : (297, 235), farthest Point : (639, 419)
Centroid : (316, 240), farthest Point : (639, 419)
Centroid : (316, 240), farthest Point : (639, 419)
Centroid : (264, 220), farthest Point : (639, 419)
Centroid : (264, 220), farthest Point : (639, 419)
Centroid : (260, 222), farthest Point : (639, 419)
Centroid : (260, 222), farthest Point : (639, 419)
Centroid : (262, 231), farthest Point : (639, 419)
Centroid : (269, 213), farthest Point : (639, 419)
Centroid : (269, 213), farthest Point : (639, 419)
Centroid : (255, 223), farthest Point : (639, 419)
Centroid : (255, 223), farthest Point : (639, 419)
Centroid : (268, 211), farthest Point : (639, 419)
Centroid : (268, 211), farthest Point : (639, 419)
Centroid : (258, 217), farthest Point : (639, 419)
Centroid : (258, 217), farthest Point : (639, 419)
Centroid : (242, 222), farthest

Centroid : (294, 233), farthest Point : (639, 419)
Centroid : (298, 234), farthest Point : (639, 419)
Centroid : (298, 234), farthest Point : (639, 419)
Centroid : (293, 217), farthest Point : (639, 419)
Centroid : (293, 217), farthest Point : (639, 419)
Centroid : (294, 235), farthest Point : (639, 419)
Centroid : (294, 235), farthest Point : (639, 419)
Centroid : (293, 235), farthest Point : (639, 419)
Centroid : (293, 235), farthest Point : (639, 419)
Centroid : (291, 236), farthest Point : (639, 419)
Centroid : (291, 236), farthest Point : (639, 419)
Centroid : (287, 233), farthest Point : (639, 419)
Centroid : (287, 233), farthest Point : (639, 419)
Centroid : (280, 216), farthest Point : (639, 419)
Centroid : (280, 216), farthest Point : (639, 419)
Centroid : (293, 217), farthest Point : (639, 419)
Centroid : (293, 217), farthest Point : (639, 419)
Centroid : (288, 218), farthest Point : (639, 419)
Centroid : (288, 218), farthest Point : (639, 419)
Centroid : (280, 217), farthest

Centroid : (266, 235), farthest Point : (639, 419)
Centroid : (266, 235), farthest Point : (639, 419)
Centroid : (263, 235), farthest Point : (639, 419)
Centroid : (263, 235), farthest Point : (639, 419)
Centroid : (266, 233), farthest Point : (639, 419)
Centroid : (266, 233), farthest Point : (639, 419)
Centroid : (243, 206), farthest Point : (639, 419)
Centroid : (243, 206), farthest Point : (639, 419)
Centroid : (263, 233), farthest Point : (639, 419)
Centroid : (263, 233), farthest Point : (639, 419)
Centroid : (258, 211), farthest Point : (639, 419)
Centroid : (258, 211), farthest Point : (639, 419)
Centroid : (256, 215), farthest Point : (639, 419)
Centroid : (256, 215), farthest Point : (639, 419)
Centroid : (243, 209), farthest Point : (639, 419)
Centroid : (243, 209), farthest Point : (639, 419)
Centroid : (252, 218), farthest Point : (639, 419)
Centroid : (252, 218), farthest Point : (639, 419)
Centroid : (245, 212), farthest Point : (639, 419)
Centroid : (245, 212), farthest

Centroid : (305, 237), farthest Point : (639, 419)
Centroid : (305, 237), farthest Point : (639, 419)
Centroid : (291, 230), farthest Point : None
Centroid : (291, 230), farthest Point : None
Centroid : (283, 225), farthest Point : None
Centroid : (283, 225), farthest Point : None
Centroid : (283, 229), farthest Point : (639, 419)
Centroid : (287, 229), farthest Point : (639, 419)
Centroid : (287, 229), farthest Point : (639, 419)
Centroid : (285, 228), farthest Point : (639, 419)
Centroid : (285, 228), farthest Point : (639, 419)
Centroid : (285, 229), farthest Point : None
Centroid : (285, 229), farthest Point : None
Centroid : (275, 224), farthest Point : (639, 419)
Centroid : (275, 224), farthest Point : (639, 419)
Centroid : (281, 226), farthest Point : (639, 419)
Centroid : (281, 226), farthest Point : (639, 419)
Centroid : (280, 225), farthest Point : (639, 419)
Centroid : (276, 227), farthest Point : (639, 419)
Centroid : (276, 227), farthest Point : (639, 419)
Centroid : (278,